# Information Retrieval

Создаем и считаем BagOfWords(bow). Записываем его в файл 'bow.pickle'. Pickle - бинарный формат хранения файла. Нечитаемый для пользователя, но читаемый для машины. 

In [ ]:
#bow = BagOfWords
N = 101
bow = set()
for i in range(N):
    filename = 'info retrieval txt/(' + str(i+1) + ').txt'
    text = open(filename, encoding='utf-8').read()
    bow = bow | set(extract_words(text))
print(len(bow))
pickle.dump(bow, open('bow.pickle', 'wb'))

Итеративный процесс: для каждого файла, содержащего новость, создаем его бинарное представление. Теперь каждый файл - это бинарный вектор. 
    Для каждого файла векторы записаны отдельно. Лежат в папке info retrieval/. 

In [ ]:
for i in range(N):
    filename = 'info retrieval txt/(' + str(i+1) + ').txt'
    text = open(filename, encoding='utf-8').read()
    words = set(extract_words(text))
    binary = {}
    for j in bow:
        binary[j] = 0
    for w in words:
        binary[w] = 1   
    #print(binary)
    file = 'info retrieval binary/' + (str(i+1)) + '.pickle'
    pickle.dump(binary, open(file, 'wb'))
#binary search     


Для каждого слова из мешка слов считаем инвертированный индекс. 
inverted_index - словарь. key = слово; value = вектор. 
Записываем в файл 'inverted_index.pickle'. 

In [ ]:
inverted_index = {}
for w in bow:
    inverted_index[w] = []
for i in range(N):
    filename = 'info retrieval txt/(' + str(i+1) + ').txt'
    text = open(filename, encoding='utf-8').read()
    words = set(extract_words(text))
    for w in words:
        inverted_index[w].append(i+1)
print(len(inverted_index))#, inverted_index)
pickle.dump(inverted_index, open('inverted_index.pickle', 'wb'))    
#inverted index

Находим документы, в которых содержатся все слова из запроса query. Если такого документа нет, возвращаем 'There is no document that corresponds to your query'. Актуально для бинарного поиска (неранжированного). 

In [ ]:
cross = set([i+1 for i in range(N)])
for q in query:
    if q in bow:
        row = set(inverted_index[q])
    else:
        cross = set()
        break
    #cross = set(inverted_index.get())
    cross = cross & row
if len(cross) == 0:
    print('There is no document that corresponds to your query')
else:
    print('Documents that correspond to you query: ', end = '')
    for i in cross:
        print(i, end = ' ')
#документы, к-е содержат ВСЕ слова из запроса (одновременно)

Считаем idf для каждого слова из мешка слов. Записываем в файл 'idf.pickle'. 

In [ ]:
#count idf
idf = {}
for word in bow:
    idf[word] = 0.0

for word in bow:
    df = len(inverted_index[word])
    local_idf = math.log10(N/df)
    idf[word] += round(local_idf, 3)
    
#print(idf)

#with open('idf.pickle', 'wb') as f:
#    pickle.dump(idf)

pickle.dump(idf, open('idf.pickle', 'wb')) 
print(idf)

# Программа

In [1]:
import pickle
import re
import math
import pymystem3
m = pymystem3.Mystem()

Функция extract_words возвращает список лемм из текста. 

In [2]:
def extract_words(text):
    lst = re.findall('\w+', text.lower())
    s = set(lst)
    lemmas = []
    for i in s:
        lemma = m.lemmatize(i)[0]
        lemmas.insert(0, lemma)
    return lemmas

Считаем tf для данного слова и документа. Т.е. считаем, сколько раз слово встречается в определенном документе. 

In [3]:
"""def count_tf(words, q):    
    result = 0
    for w in words:
        if w == q:
            result += 1
    return result
"""
def count_tf(words, q):
    text = ' '.join([w for w in words])
    #print(text)
    raw = text.count(q)
    if not raw:
        return 0
    return raw

Загружаем ранее подсчитанные величины: мешок слов, обратный индекс, обратную частоту документа. 

In [4]:
bow = pickle.load(open('bow.pickle', 'rb'))
inverted_index = pickle.load(open('inverted_index.pickle', 'rb'))
idf = pickle.load(open('idf.pickle', 'rb'))
#idf

Вводим запрос и лемматизируем его. 

In [5]:
#query = input().lower()
Q = 'Россия недовольна Меланией'
query = Q.lower()
print(query)
query = extract_words(query)
print(query)

россия недовольна меланией
['россия', 'недовольный', 'мелания']


Считаем максимальный TF-IDF, отвечаем на запрос. 

In [7]:
print('Query is:', Q)

tf_idf = {}
for q in query:
    #print(q)
    if q not in bow:
        continue
    
    for i in inverted_index[q]:
        filename = 'info retrieval txt/(' + str(i) + ').txt'
        text = open(filename, encoding='utf-8').read()
        words = extract_words(text)
        tf = 1.0 + math.log10(count_tf(words, q))
        if i in tf_idf:
            tf_idf[i] +=tf * idf[q]
        else:
            tf_idf[i] = tf * idf[q]      
#print(tf_idf)
doc = 0
m = 0.0
for key in tf_idf:
    if tf_idf[key] > m:
        m = round(tf_idf[key], 3)
        doc = key

if doc == 0:
    print('There is no document that corresponds to your query')
else:
    print('Document number {0} corresponds to query the most'.format(doc))
    print('TF-IDF weight equals {0}'.format(m))

Query is: Россия недовольна Меланией
Document number 64 corresponds to query the most
TF-IDF weight equals 0.63


In [ ]:
#[print(i, end=' ') for i in Q]

In [ ]:
#'\w+|\S'
#'\S' = НЕ white space

In [ ]:
"""if t not in d:
    d[t] = []
d[t].append(i)

with open('inverted_index.pickle', 'wb') as f:
    pickle.dump(d, f)

str = ' '.join([str(id) for id in d[t]])
"""

In [ ]:
#res = set(d.get(terms[0]))

In [ ]:
"""for w, f in all_idfs.items():
    idf = math.log(N/f)
    all_idfs[w] = idf
    
!работает, т.к. идем не по all_idfs, а по.items()"""